# ЦЕЛЬ

Исследовать поведение пользователей в обновлённом приложении.

# ЗАДАЧИ

- Понять, как пользователи взаимодействуют с продуктом, и соотнести идеальный путь пользователей с фактическим.
- Выявить этапы, которые занимают больше всего времени, с тем чтобы поработать над их улучшением.
- Проанализировать зависимость оплат от прохождения обучения.

# КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)

- Определить самые распространённые пути прохождения (последовательности) этапов в приложении.
- Посмотреть на среднее время между различными этапами и выделить самые большие временные промежутки.
- Определить, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:
    * пользователями, которые прошли обучение хотя бы раз;
    * пользователями, которые начали обучение, но не прошли его ни разу;
    * пользователями, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

In [1]:
import pandas as pd

## Таблица *Event*
    Хранит данные о событиях, которые совершают пользователи. По сути, каждое событие — это факт прохождения пользователем какого-либо этапа игры.

    Название поля	 Описание
    id	            идентификатор события
    user_id	       уникальный идентификатор пользователя, совершившего событие в приложении
    start_time	    дата и время события
    event_type	    тип события 
                (значения: registration — регистрация; 
                tutorial_start — начало обучения; 
                tutorial_finish — завершение обучения; 
                level_choice — выбор уровня сложности; 
                pack_choice — выбор пакетов вопросов)
    tutorial_id	   идентификатор обучения (этот идентификатор есть только у событий обучения)
    selected_level	выбранный уровень сложности обучения

In [2]:
events = pd.read_csv('Events.csv', sep=',')
events

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
...,...,...,...,...,...,...
252329,281232,level_choice,hard,2020-07-02T10:02:15,NaN,87439
252330,281233,level_choice,medium,2020-07-02T11:38:52,NaN,87488
252331,281234,pack_choice,NaN,2020-07-02T11:42:14,NaN,87488
252332,281235,tutorial_start,NaN,2020-07-02T13:32:58,86127.0,87464


In [3]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


In [4]:
events_2018 = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[events_2018]['user_id'].to_list() # список пользователей, зарег. в 2018
events_df = events[events.user_id.isin(registered)]

In [5]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              66959 non-null  int64  
 1   event_type      66959 non-null  object 
 2   selected_level  8342 non-null   object 
 3   start_time      66959 non-null  object 
 4   tutorial_id     32954 non-null  float64
 5   user_id         66959 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.6+ MB


In [6]:
events_df['start_time'] = pd.to_datetime(events_df['start_time'])

C:\Users\6145~1\AppData\Local\Temp/ipykernel_6852/2881646811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['start_time'] = pd.to_datetime(events_df['start_time'])


In [7]:
events_df[events_df['event_type'] == 'level_choice'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 51424 to 118363
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8342 non-null   int64         
 1   event_type      8342 non-null   object        
 2   selected_level  8342 non-null   object        
 3   start_time      8342 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         8342 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 456.2+ KB


In [8]:
events_df[events_df['event_type'].isin(['tutorial_start','tutorial_finish'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32954 entries, 51412 to 118360
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              32954 non-null  int64         
 1   event_type      32954 non-null  object        
 2   selected_level  0 non-null      object        
 3   start_time      32954 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         32954 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.8+ MB


In [9]:
events_df.describe(include='all')

C:\Users\6145~1\AppData\Local\Temp/ipykernel_6852/2649091955.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  events_df.describe(include='all')


,id,event_type,selected_level,start_time,tutorial_id,user_id
count,66959.000000,66959,8342,66959,32954.000000,66959.000000
unique,NaN,5,3,66809,NaN,NaN
top,NaN,registration,medium,2018-03-13 09:37:43,NaN,NaN
freq,NaN,19926,4645,9,NaN,NaN
first,NaN,NaN,NaN,2018-01-01 03:48:40,NaN,NaN
last,NaN,NaN,NaN,2019-01-01 05:50:36,NaN,NaN
mean,113787.000045,NaN,NaN,NaN,40532.934393,37781.543362
std,19329.542752,NaN,NaN,NaN,5213.486632,5751.497904
min,80308.000000,NaN,NaN,NaN,31505.000000,27832.000000
25%,97047.500000,NaN,NaN,NaN,36008.250000,32849.000000


In [10]:
events_df['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

In [11]:
events_df['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [12]:
events_df['user_id'].nunique()

19926

## Таблица *purchase*
    Хранит данные об оплатах, которые совершают пользователи.

    Название поля	 Описание
    id	            идентификатор события
    user_id	       уникальный идентификатор пользователя, совершившего событие в приложении
    event_datetime	дата и время события/покупки
    amount	        сумма оплаты

In [13]:
purchase = pd.read_csv('purchase.csv', sep=',')
purchase

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
...,...,...,...,...
5951,21625,87331,2020-07-06T09:02:07,50
5952,21626,87418,2020-07-06T14:16:37,100
5953,21627,87431,2020-07-06T22:48:59,50
5954,21628,87363,2020-07-07T05:38:56,100


In [14]:
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


In [15]:
purchase['event_datetime'] = pd.to_datetime(purchase['event_datetime'])

In [16]:
purchase_df = purchase[purchase.user_id.isin(registered)]
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


In [17]:
purchase_df.describe()

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


# АНАЛИЗ ПОЛЬЗОВАТЕЛЬСКИХ СОБЫТИЙ

Для того чтобы понимать, как пользователи переходят из этапа в этап и на каких из них возникают сложности, мы должны определить конверсию на каждом из этапов воронки. То есть нам нужно понять, какой процент пользователей переходит с предыдущего этапа на следующий.

## Изучаем событие registration

In [18]:
events_df[events_df['event_type'] == 'registration']['user_id'].nunique() # кол-во пользователей свершившие событие registration

19926

In [19]:
events_df['user_id'].nunique() # общее кол-во пользователей

19926

Как мы видим, все пользователи совершают событие registration. Это обусловлено тем, что данный этап является обязательным для выполнения и без него пользователь не сможет перейти дальше.

## Изучаем событие tutorial_start

In [20]:
events_df[events_df['event_type'] == 'tutorial_start'].head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
51412,80315,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836
51413,80316,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835
51415,80318,tutorial_start,NaN,2018-01-01 15:40:43,31507.0,27836
51417,80320,tutorial_start,NaN,2018-01-01 17:47:40,31508.0,27833
51420,80323,tutorial_start,NaN,2018-01-01 19:11:36,31509.0,27839
51422,80325,tutorial_start,NaN,2018-01-01 19:46:11,31510.0,27834
51434,80337,tutorial_start,NaN,2018-01-02 02:07:07,31511.0,27840
51435,80338,tutorial_start,NaN,2018-01-02 03:03:44,31512.0,27845
51437,80340,tutorial_start,NaN,2018-01-02 04:55:11,31513.0,27842
51446,80349,tutorial_start,NaN,2018-01-02 07:08:00,31514.0,27845


In [21]:
events_df[events_df['event_type'] == 'tutorial_start']['user_id'].nunique() # кол-во пользователей начавших обучение

11858

In [22]:
registered_users_count = events_df[events_df["event_type"] == "registration"][
    "user_id"
].nunique()
tutorial_start_users_count = events_df[events_df["event_type"] == "tutorial_start"][
    "user_id"
].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count
print(
    "Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_tutorial_start_users
    )
)

Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): 59.51%


Как мы видим, число пользователей, которые перешли к выполнению обучения, меньше, чем число пользователей, прошедших регистрацию.

## Изучаем событие tutorial_finish

In [23]:
 events_df[events_df['event_type'] == 'tutorial_finish']['user_id'].nunique() # кол-во пользователей заершивших обучение

10250

In [24]:
tutorial_finish_users_count = events_df[events_df["event_type"] == "tutorial_finish"][
    "user_id"
].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count
print(
    "Процент пользователей, завершивших обучение: {:.2%}".format(
        tutorial_completion_rate
    )
)

Процент пользователей, завершивших обучение: 86.44%


В нашем приложении достаточно хороший процент прохождения обучения. Но есть куда стремиться для его увеличения.

## Изучаем level_choice

In [25]:
events_df[events_df['event_type'] == 'level_choice']['user_id'].nunique() # кол-во пользователей выбравших уровень сложности

8342

In [26]:
level_choice_users_count = events_df[events_df["event_type"] == "level_choice"][
    "user_id"
].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print(
    "Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_level_choice_users
    )
)

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): 41.86%


Меньше половины пользователей (41.86 %) доходят до этапа выбора уровня сложности вопросов. А ведь этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные возможности, которые в дальнейшем могут привести к оплате.

Таким образом, для успешной монетизации приложения крайне важно оптимизировать прохождение до этапа выбора сложности.

## Изучаем pack_choice

In [27]:
events_df[events_df['event_type'] == 'pack_choice']['user_id'].nunique() # кол-во пользователей, которые совершили событие выбора бесплатного пакета вопросов

5737

In [28]:
training_choice_users_count = events_df[events_df["event_type"] == "pack_choice"][
    "user_id"
].nunique()
percent_training_choice_users = training_choice_users_count / level_choice_users_count
print(
    "Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_training_choice_users
    )
)

Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): 68.77%


## Изучаем покупку платных пакетов

In [29]:
purchase_df['user_id'].nunique() # кол-во пользователей свершивших покупку

1600

In [30]:
paying_users_count = purchase_df["user_id"].nunique()
percent_of_paying_users = paying_users_count / training_choice_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): 27.89%


In [31]:
purchase_rate = paying_users_count / registered_users_count
print(
    "Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): {:.2%}".format(
        purchase_rate
    )
)

Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): 8.03%


Процент зарегистрировавшихся пользователей, которые приобрели платный пакет, составляет 8.03 %.

Само по себе это число нам ни о чём не говорит, поэтому в рамках одной из гипотез мы рассмотрим этот процент для пользователей разных групп. И эта информация укажет нам возможные пути оптимизации.

## АНАЛИЗ УНИКАЛЬНЫХ ПОЛЬЗОВАТЕЛЬСКИХ ПУТЕЙ

#### Зачем нужны уникальные пользовательские пути
Наша воронка достаточно предсказуемая: почти все этапы идут последовательно друг за другом. Однако нужно иметь в виду один нюанс: есть пользователи, которые не начинают обучение, есть такие, кто его не заканчивает, а есть те, кто проходят его по несколько раз.

Отследив уникальные пользовательские пути, мы можем визуально оценить, какие успешные пути существуют, а также заметить девиации (например, если у нас будет много путей с очень частым началом обучения).

Чтобы понимать, какие есть различные последовательности прохождения пользователей по воронке и насколько часто они встречаются, можно воспользоваться таким подходом:

- отсортировать все события по возрастанию во времени;
- объединить для каждого пользователя все его события в один список;
- подсчитать частоту различных списков.

In [32]:
purchase_df['event_type'] = 'purchase' # Добавляем в датафрейм purchase_df столбец event_type, который будет содержать одно значение purchase.
                                       # Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.

C:\Users\6145~1\AppData\Local\Temp/ipykernel_6852/4271691521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_df['event_type'] = 'purchase' # Добавляем в датафрейм purchase_df столбец event_type, который будет содержать одно значение purchase.


Также у нас есть одинаковые столбцы id в двух датафреймах, но смысл их несколько отличается, так как столбец id в events_df указывает на идентификатор события, а столбец id в purchase_df указывает на идентификатор оплаты. Поэтому применим функцию rename(), чтобы переименовать столбцы в датафреймах.

In [33]:
events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})

In [34]:
total_events_df = pd.concat([events_df,purchase_df],sort=False) # объединение датафреймов

In [35]:
total_events_df.head(10)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
51410,80313.0,registration,NaN,2018-01-01 14:08:40,NaN,27837,NaN,NaT,NaN
51411,80314.0,registration,NaN,2018-01-01 14:42:58,NaN,27838,NaN,NaT,NaN
51412,80315.0,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836,NaN,NaT,NaN
51413,80316.0,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaN,NaT,NaN
51414,80317.0,tutorial_finish,NaN,2018-01-01 15:06:15,31506.0,27835,NaN,NaT,NaN


In [36]:
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


In [37]:
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time') # сброс индексов + сортировка событий по возрастанию времени

### Воссоздаём последовательность событий

In [38]:
user_path_df = (
    total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
)
user_path_df.head(10) # формирование для каждого пользователя список, содержащий во временной последовательности все события, совершаемые данным пользователем

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


### Выявляем наиболее популярные пути

In [39]:
user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))
user_path_df["event_path"].head()    #  преобразование списока событий в строку

0                                         registration
1      registration > tutorial_start > tutorial_finish
2      registration > tutorial_start > tutorial_finish
3    registration > tutorial_start > tutorial_finis...
4    registration > tutorial_start > tutorial_start...
Name: event_path, dtype: object

In [40]:
user_paths = (
    user_path_df.groupby(["event_path"])["user_id"]
    .nunique()
    .sort_values(ascending=False)
)

In [41]:
user_paths.head(10)

event_path
registration                                                                                                       7970
registration > tutorial_start > tutorial_finish > level_choice > pack_choice                                       2796
registration > tutorial_start > tutorial_finish                                                                    1956
registration > tutorial_start > tutorial_finish > level_choice                                                     1713
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                            1083
registration > tutorial_start                                                                                       842
registration > tutorial_start > level_choice > pack_choice                                                          346
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish                                  323
registration > tutorial_start

Как мы видим, среди 10 самых популярных последовательностей только одна содержит этап оплаты. Это последовательность registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase.

### Ищем последовательности с оплатой

In [42]:
user_paths[user_paths.index.str.contains('purchase')].head(10)

event_path
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                                             1083
registration > tutorial_start > level_choice > pack_choice > purchase                                                                                                                                124
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > tutorial_start > tutorial_finish > purchase                                                                           101
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                            52
registration > tutorial_start > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                 

Большинство последовательностей, которые содержат в себе оплату, также содержат старт обучения. Это наводит нас на гипотезу, что вероятность оплаты зависит от того, проходил ли пользователь обучение.

## АНАЛИЗ ВРЕМЕННЫХ ПРОМЕЖУТКОВ

Этот шаг очень полезен при анализе поведения пользователей. Время между этапами может сообщить нам о вовлечённости пользователей в приложение, а также о роли отдельных этапов на развитие этой вовлечённости. Если окажется, что у пользователей, которые проходят обучение, время до выбора вопросов меньше, чем у тех, кто его не проходит, то разработчикам нужно будет придумывать способы, как вовлекать пользователя в прохождение данного этапа.

В целом, чем меньше срок до целевого действия (оплаты) в среднем, тем более управляемо наше приложение, и, скорее всего, на пути пользователя будет меньше отвлекающих факторов, которые ему помешают совершить оплату.

### Определяем время между registration и tutorial_start

##### 1 - registration

In [43]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration'] # выделяем отдельный датафрейм registration_df, содержащий только события с event_type = registration. 
                                                                                   # Этот датафрейм будет вспомогательным для определения времени между регистрацией и началом обучения

In [44]:
registration_df['user_id'].value_counts().mean() # Проверка, что в датафрейме registration_df для каждого пользователя содержится только по одной регистрации на пользователя.

1.0

In [45]:
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
) # Оставляем только нужные для вычислений столбцы

##### 2 - tutorial_start

In [46]:
tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start'] #  Выделяем отдельный датафрейм tutorial_start_df, содержащий только события с event_type = tutorial_start (начало обучения)

In [47]:
tutorial_start_df['user_id'].value_counts().mean() # Расчет сколько таких событий приходится на пользователя

1.522179119581717

У некоторых наших пользователей более чем по одному событию начала обучения. Оставим только первое обучение. Таким образом мы будем искать время между регистрацией и началом первого обучения пользователя.

In [48]:
tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values(
    "start_time"
).drop_duplicates("user_id")           # Создаем датафрейм, содержащий только первое обучение

In [49]:
tutorial_start_df_wo_duplicates['user_id'].value_counts().mean()    # Проверка сколько таких событий приходится на пользователя

1.0

In [50]:
tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[
    ["user_id", "tutorial_id", "start_time"]
].rename(columns={"start_time": "tutorial_start_time"})      # Оставляем только нужные для вычислений столбцы

In [51]:
merged_df = registration_df.merge(
    tutorial_start_df_wo_duplicates, on="user_id", how="inner"
)
merged_df.head()

,user_id,registration_time,tutorial_id,tutorial_start_time
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36


In [52]:
merged_df["timedelta"] = (
    merged_df["tutorial_start_time"] - merged_df["registration_time"]
)
merged_df.head()        # Считаем временную разницу

,user_id,registration_time,tutorial_id,tutorial_start_time,timedelta
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40,0 days 13:40:15
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11,0 days 11:11:01
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51,0 days 03:06:04
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40,0 days 01:26:33
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36,0 days 00:47:35


In [53]:
merged_df["timedelta"].mean() # среднее время, которое проходит между регистрацией пользователя и началом первого обучения

Timedelta('0 days 04:38:24.019817844')

In [54]:
merged_df['timedelta'].describe()

count                        11858
mean     0 days 04:38:24.019817844
std      0 days 04:15:09.650705034
min                0 days 00:00:34
25%         0 days 01:21:37.500000
50%         0 days 03:22:08.500000
75%         0 days 06:47:42.750000
max                1 days 16:03:46
Name: timedelta, dtype: object

Первый квартиль составляет 1 час 21 минут. А медиана (второй квартиль) — 3 часа 22 минут.

Таким образом:
- четверть пользователей тратит меньше 1 часа 21 минут на переход от регистрации к началу обучения;
- половина всех пользователей тратит между регистрацией и началом обучения менее 3 часов 22 минут.

Такая сегментация пользователей может дать нам важные опорные точки для оптимизации времени. Мы можем сосредоточится не на оптимизации среднего времени (поскольку среднее будет увеличено за счёт пользователей, которые долго проходят этап), а на том, чтобы оптимизировать медианное время (время, которое тратит половина пользователей).

Также мы можем отдельно выделить тех пользователей, у которых время больше медианного, чтобы исследовать их детально и понять, что мешает им быстрее дойти до этапа обучения.

##### Измеряем время обучения

In [55]:
tutorial_finish_df = total_events_df[total_events_df["event_type"] == "tutorial_finish"] # датафрейм содержащий события окончания обучения.

In [56]:
first_tutorial_ids = tutorial_start_df_wo_duplicates["tutorial_id"].unique() # список с идентификаторами первых обучений - уникальные значения столбца tutorial_id в уже созданном датафрейме tutorial_start_df_wo_duplicates.

In [57]:
tutorial_finish_df = tutorial_finish_df[
    tutorial_finish_df["tutorial_id"].isin(first_tutorial_ids)
]                                                               # фильтруем датафрейм, оставляем только события для обучений, которые были первыми для пользователя по идентификаторам

In [58]:
tutorial_finish_df['user_id'].value_counts().mean()         # Проверка сколько таких событий приходится на пользователя

1.0

In [59]:
tutorial_finish_df = tutorial_finish_df[["user_id", "start_time"]].rename(
    columns={"start_time": "tutorial_finish_time"}
)                                                       # Оставляем только нужные для вычислений столбцы

In [60]:
merged_df_2 = tutorial_start_df_wo_duplicates.merge(
    tutorial_finish_df, on="user_id", how="inner"
)                                                      # объединение таблиц

In [61]:
merged_df_2["timedelta"] = (
    merged_df_2["tutorial_finish_time"] - merged_df_2["tutorial_start_time"]
)
merged_df_2.head()                              # Считаем временную разницу

,user_id,tutorial_id,tutorial_start_time,tutorial_finish_time,timedelta
0,27835,31506.0,2018-01-01 15:00:51,2018-01-01 15:06:15,0 days 00:05:24
1,27833,31508.0,2018-01-01 17:47:40,2018-01-01 17:50:08,0 days 00:02:28
2,27839,31509.0,2018-01-01 19:11:36,2018-01-01 19:16:32,0 days 00:04:56
3,27834,31510.0,2018-01-01 19:46:11,2018-01-01 19:48:01,0 days 00:01:50
4,27845,31512.0,2018-01-02 03:03:44,2018-01-02 03:06:48,0 days 00:03:04


In [62]:
merged_df_2["timedelta"].mean() # среднее время, которое проходит первое обучение

Timedelta('0 days 00:03:53.174160732')

In [63]:
merged_df_2["timedelta"].describe()

count                         9830
mean     0 days 00:03:53.174160732
std      0 days 00:01:44.803217992
min                0 days 00:00:16
25%                0 days 00:02:28
50%                0 days 00:03:42
75%                0 days 00:05:08
max                0 days 00:10:06
Name: timedelta, dtype: object

##### Определяем время между registration и level_choice

In [64]:
level_choice_df = total_events_df[total_events_df["event_type"] == "level_choice"] # датафрейм содержащий события выбора уровня.

In [65]:
level_choice_df["user_id"].value_counts().mean() # среднее на одного пользователя

1.0

In [66]:
level_choice_df = level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"})                # Оставляем только нужные для вычислений столбцы

In [67]:
merged_df_3 = registration_df.merge(level_choice_df, on="user_id", how="inner")
merged_df_3["timedelta"] = (
    merged_df_3["level_choice_time"] - merged_df_3["registration_time"]
)
merged_df_3.head()               # Считаем временную разницу

,user_id,registration_time,level_choice_time,timedelta
0,27835,2018-01-01 11:54:47,2018-01-01 20:37:22,0 days 08:42:35
1,27839,2018-01-01 18:24:01,2018-01-01 22:37:50,0 days 04:13:49
2,27840,2018-01-01 20:53:21,2018-01-02 05:18:42,0 days 08:25:21
3,27842,2018-01-01 23:18:46,2018-01-02 08:46:03,0 days 09:27:17
4,27843,2018-01-02 00:02:28,2018-01-02 14:09:58,0 days 14:07:30


In [68]:
merged_df_3["timedelta"].mean() # среднее время, которое проходит между выбором уровня пользователя и регистрацией


Timedelta('0 days 07:10:19.169863342')

In [69]:
merged_df_3["timedelta"].describe()

count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object

##### Определяем время между level_choice и pack_choice

In [70]:
pack_choice_df = total_events_df[total_events_df["event_type"] == "pack_choice"]  # Создаем датафрейм, содержащий только выбор пакета вопросов

In [71]:
pack_choice_df["user_id"].value_counts().mean()     # Проверка сколько таких событий приходится на пользователя

1.0

In [72]:
pack_choice_df = pack_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "pack_choice_time"}
)                                                   # Оставляем только нужные для вычислений столбцы

In [73]:
merged_df_4 = level_choice_df.merge(pack_choice_df, on="user_id", how="inner")

In [74]:
merged_df_4["timedelta"] = (
    merged_df_4["pack_choice_time"] - merged_df_4["level_choice_time"]
)
merged_df_4.head()                # Считаем временную разницу

,user_id,level_choice_time,pack_choice_time,timedelta
0,27835,2018-01-01 20:37:22,2018-01-01 20:38:43,0 days 00:01:21
1,27839,2018-01-01 22:37:50,2018-01-01 22:42:54,0 days 00:05:04
2,27845,2018-01-02 06:19:18,2018-01-02 06:25:12,0 days 00:05:54
3,27849,2018-01-02 11:53:11,2018-01-02 11:59:26,0 days 00:06:15
4,27843,2018-01-02 14:09:58,2018-01-02 14:14:51,0 days 00:04:53


In [75]:
merged_df_4["timedelta"].mean() # среднее время, которое проходит между выбором уровня и выбором пакета вопросов

Timedelta('0 days 00:05:17.128464354')

In [76]:
merged_df_4["timedelta"].describe()

count                         5737
mean     0 days 00:05:17.128464354
std      0 days 00:02:43.923107804
min                0 days 00:00:19
25%                0 days 00:03:05
50%                0 days 00:04:57
75%                0 days 00:07:08
max                0 days 00:15:48
Name: timedelta, dtype: object

##### Определяем время между pack_choice и purchase

In [77]:
purchase_first_df = total_events_df[total_events_df["event_type"] == "purchase"]  # Создаем датафрейм, содержащий только первую покупку

In [78]:
purchase_first_df["user_id"].value_counts().mean()            # Проверка сколько таких событий приходится на пользователя

1.0

In [79]:
purchase_first_df = purchase_first_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)            # Оставляем только нужные для вычислений столбцы

In [80]:
merged_df_5 = pack_choice_df.merge(purchase_first_df, on="user_id", how="inner")

In [81]:
merged_df_5["timedelta"] = (
    merged_df_5["purchase_time"] - merged_df_5["pack_choice_time"]
)           # Считаем временную разницу

In [82]:
merged_df_5

,user_id,pack_choice_time,purchase_time,timedelta
0,27845,2018-01-02 06:25:12,2018-01-03 18:53:43,1 days 12:28:31
1,27865,2018-01-04 06:03:20,2018-01-04 14:46:10,0 days 08:42:50
2,27884,2018-01-04 16:22:03,2018-01-08 19:37:34,4 days 03:15:31
3,27910,2018-01-05 12:05:28,2018-01-07 12:11:34,2 days 00:06:06
4,27911,2018-01-05 17:40:37,2018-01-07 08:19:12,1 days 14:38:35
...,...,...,...,...
1595,47659,2018-12-29 05:14:54,2018-12-31 17:13:54,2 days 11:59:00
1596,47687,2018-12-29 11:41:30,2018-12-30 09:05:28,0 days 21:23:58
1597,47712,2018-12-29 22:48:15,2018-12-31 09:05:20,1 days 10:17:05
1598,47742,2018-12-31 05:06:10,2019-01-04 12:51:41,4 days 07:45:31


In [83]:
merged_df_5["timedelta"].mean() # среднее время, которое проходит между выбором пакетов вопросов и первой покупкой

Timedelta('3 days 17:46:53.403125')

In [84]:
merged_df_5["timedelta"].describe()

count                         1600
mean        3 days 17:46:53.403125
std      2 days 04:37:20.225124289
min                0 days 00:44:50
25%         1 days 21:24:13.250000
50%         3 days 12:51:25.500000
75%         5 days 09:42:13.750000
max               10 days 18:33:59
Name: timedelta, dtype: object

## Проверка аналитической гипотезы

Определим, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:

- пользователи, которые прошли обучение хотя бы раз;
- пользователи, которые начали обучение, но не прошли его ни разу;
- пользователи, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

### Ищем пользователей, прошедших обучение хотя бы раз

In [85]:
users_with_finished_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_finish"
]["user_id"].unique()
print(len(users_with_finished_tutorial))

10250


### Ищем пользователей, которые начали, но не закончили обучение

In [86]:
users_with_started_tutorial = total_events_df[
    total_events_df["event_type"] == "tutorial_start"
]["user_id"].unique()  # все пользователи начавшие обучение

In [87]:
set_users_with_started_tutorial = set(users_with_started_tutorial)
set_users_not_finished_but_started_tutorial = (
    set_users_with_started_tutorial.difference(set(users_with_finished_tutorial))
)

In [88]:
len(set_users_with_started_tutorial) # кол-во пользователей начавших обучение

11858

In [89]:
len(set_users_not_finished_but_started_tutorial) # кол-во пользователей незакончивших обучение

1608

### Ищем пользователей, сразу выбравших уровень сложности

In [90]:
all_users = total_events_df["user_id"].unique()  # общее количество пользователей

In [91]:
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(
    set_users_with_started_tutorial
)

In [92]:
len(set_users_not_started_tutorial) # кол-ко пользователей неначавшие обучение

8068

Проверим, что мы правильно разбили пользователей на группы.

Просуммировав длину всех множеств пользователей, мы должны получить исходное количество всех пользователей

In [93]:
len(set_users_not_finished_but_started_tutorial) + len(set_users_not_started_tutorial) + len(users_with_finished_tutorial) == len(set_all_users)

True

### Считаем пользователей, которые завершили обучение и совершили оплату

In [94]:
purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_with_finished_tutorial)] # датафрейм, содержащий данные по оплатам пользователей, которые завершили обучение

In [95]:
purchase_df_1['user_id'].nunique() #кол-во пользователей с оплатой и пройденным обучением

1447

In [96]:
percent_of_purchase_1 = purchase_df_1["user_id"].nunique() / len(
    users_with_finished_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): {:.2%}".format(
        percent_of_purchase_1
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): 14.12%


### Считаем средний размер платежа разных групп

In [97]:
purchase_df_1['amount'].mean() # средний размер платежа у пользователей, которые завершили обучение

110.98825155494126

оплаты пользователей, начавших, но не закончивших обучение

In [98]:
purchase_df_2 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_finished_but_started_tutorial)
]
print(purchase_df_2["user_id"].nunique())
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(
    set_users_not_finished_but_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): {:.2%}".format(
        percent_of_purchase_2
    )
)

131
Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): 8.15%


In [99]:
purchase_df_2['amount'].mean() # средний размер платежа у пользователей, которые не завершили обучение

104.9618320610687

оплаты пользователей, не начинавших обучение

In [100]:
purchase_df_3 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_started_tutorial)
]
print(purchase_df_3["user_id"].nunique())
percent_of_purchase_3 = purchase_df_3["user_id"].nunique() / len(
    set_users_not_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, не начавших обучение): {:.2%}".format(
        percent_of_purchase_3
    )
)

22
Процент пользователей, которые оплатили тренировки (от числа пользователей, не начавших обучение): 0.27%


In [101]:
purchase_df_3['amount'].mean() # средний размер платежа у пользователей, которые не начинали обучение

128.4090909090909

Как мы видим, процент пользователей, которые завершили обучение и совершили оплату, выше, чем процент пользователей, которые не начали обучение или не закончили его.

Это говорит о том, что факт прохождения обучения влияет на дальнейшую мотивацию использовать приложение. Существенной разницы в среднем чеке при этом не наблюдается, то есть обучение увеличивает вовлечённость, но выбор количества платных пакетов зависит индивидуально от пользователя.

# ***  Самостоятельная работа ***

## ЗАДАНИЕ
Необходимо проверить:

- есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
- различается ли временной промежуток между выбором уровня сложности и оплатой у групп пользователей с разным уровнем сложности
- различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

### Зависимость между выбранным уровнем сложности и вероятностью оплаты

#### Уровень сложности easy

In [102]:
selected_level_easy_df = total_events_df[total_events_df["selected_level"] == "easy"] # Создаем датафрейм, с выбором уровня "easy"

In [103]:
selected_level_easy_df["user_id"].value_counts().mean() # Проверка сколько таких событий приходится на пользователя

1.0

##### Ищем пользователей с уровнем сложности easy

In [104]:
selected_level_easy_users = selected_level_easy_df["user_id"].unique()
len(selected_level_easy_users) # кол-во пользователей с уровнем easy

2448

##### Оплата от пользователей с уровнем сложности easy

In [105]:
purchase_df_easy = purchase_df[purchase_df["user_id"].isin(selected_level_easy_users)] # датафрейм с оплатой

In [106]:
purchase_df_easy['user_id'].nunique() #кол-во пользователей с оплатой  и уровнем easy

189

In [107]:
percent_of_purchase_easy = purchase_df_easy["user_id"].nunique() / len(
    selected_level_easy_users
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности easy): {:.2%}".format(
        percent_of_purchase_easy
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности easy): 7.72%


In [108]:
purchase_df_easy['amount'].mean() # средний размер оплаты

114.94708994708995

#### Уровень сложности medium

In [109]:
selected_level_medium_df = total_events_df[total_events_df["selected_level"] == "medium"] # Создаем датафрейм, с выбором уровня "medium"

In [110]:
selected_level_medium_df["user_id"].value_counts().mean() # Проверка сколько таких событий приходится на пользователя

1.0

##### Ищем пользователей с уровнем сложности medium

In [111]:
selected_level_medium_users = selected_level_medium_df["user_id"].unique()
len(selected_level_medium_users) # кол-во пользователей с уровнем medium

4645

##### Оплата от пользователей с уровнем сложности medium

In [112]:
purchase_df_medium = purchase_df[purchase_df["user_id"].isin(selected_level_medium_users)] # датафрейм с оплатой

In [113]:
purchase_df_medium['user_id'].nunique() #кол-во пользователей с оплатой  и уровнем medium

969

In [114]:
percent_of_purchase_medium = purchase_df_medium["user_id"].nunique() / len(
    selected_level_medium_users
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности medium): {:.2%}".format(
        percent_of_purchase_medium
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности medium): 20.86%


In [115]:
purchase_df_medium['amount'].mean() # средний размер оплаты

109.52012383900929

#### Уровень сложности hard

In [116]:
selected_level_hard_df = total_events_df[total_events_df["selected_level"] == "hard"] # Создаем датафрейм, с выбором уровня "hard"

In [117]:
selected_level_hard_df["user_id"].value_counts().mean() # Проверка сколько таких событий приходится на пользователя

1.0

##### Ищем пользователей с уровнем сложности hard

In [118]:
selected_level_hard_users = selected_level_hard_df["user_id"].unique()
len(selected_level_hard_users) # кол-во пользователей с уровнем hard

1249

##### Оплата от пользователей с уровнем сложности hard

In [119]:
purchase_df_hard = purchase_df[purchase_df["user_id"].isin(selected_level_hard_users)] # датафрейм с оплатой

In [120]:
purchase_df_hard['user_id'].nunique() #кол-во пользователей с оплатой  и уровнем easy

442

In [121]:
percent_of_purchase_hard = purchase_df_hard["user_id"].nunique() / len(
    selected_level_hard_users
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности hard): {:.2%}".format(
        percent_of_purchase_hard
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности hard): 35.39%


In [122]:
purchase_df_hard['amount'].mean() # средний размер оплаты

111.59502262443439

- Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности easy): 7.72% 
- Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности medium): 20.86%
- Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности hard): 35.39%

Как можно заметить процент процент пользователей, которые оплатили тренировки растёт с повышение уровня.
Так процент пользователей, которые оплатили тренировки с выбранным уровнем сложности hard в 4,6 раз выше чем у пользователей с уровнем easy

### Расчет временного промежутка между выбором уровня сложности и оплатой у групп пользователей с разным уровнем сложности

#### Уровень сложности easy

In [123]:
selected_level_easy_df_1 = selected_level_easy_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"})                # Оставляем только нужные для вычислений столбцы


In [124]:
selected_level_1 = selected_level_easy_df_1.merge(purchase_first_df, on="user_id", how="inner")


In [125]:
selected_level_1["timedelta"] = (
    selected_level_1["purchase_time"] - selected_level_1["level_choice_time"]
)
selected_level_1.head()               # Считаем временную разницу

,user_id,level_choice_time,purchase_time,timedelta
0,27884,2018-01-04 16:18:39,2018-01-08 19:37:34,4 days 03:18:55
1,28090,2018-01-09 21:34:23,2018-01-15 23:42:55,6 days 02:08:32
2,28182,2018-01-11 18:44:45,2018-01-12 02:46:01,0 days 08:01:16
3,28207,2018-01-11 21:10:51,2018-01-12 21:00:24,0 days 23:49:33
4,28254,2018-01-12 16:48:24,2018-01-19 22:08:40,7 days 05:20:16


In [126]:
selected_level_1["timedelta"].describe()

count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object

#### Уровень сложности medium

In [127]:
selected_level_medium_df_1 = selected_level_medium_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"})                # Оставляем только нужные для вычислений столбцы


In [128]:
selected_level_2 = selected_level_medium_df_1.merge(purchase_first_df, on="user_id", how="inner")


In [129]:
selected_level_2["timedelta"] = (
    selected_level_2["purchase_time"] - selected_level_2["level_choice_time"]
)
selected_level_2.head()               # Считаем временную разницу

,user_id,level_choice_time,purchase_time,timedelta
0,27973,2018-01-07 05:29:30,2018-01-13 21:50:00,6 days 16:20:30
1,27981,2018-01-07 10:46:14,2018-01-07 23:20:25,0 days 12:34:11
2,28010,2018-01-08 00:00:52,2018-01-10 05:32:47,2 days 05:31:55
3,28020,2018-01-08 14:47:35,2018-01-11 21:43:03,3 days 06:55:28
4,28033,2018-01-08 17:06:39,2018-01-16 05:08:41,7 days 12:02:02


In [130]:
selected_level_2["timedelta"].describe()

count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16:07:19
max               10 days 13:51:01
Name: timedelta, dtype: object

#### Уровень сложности hard

In [131]:
selected_level_hard_df_1 = selected_level_hard_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"})                # Оставляем только нужные для вычислений столбцы


In [132]:
selected_level_3 = selected_level_hard_df_1.merge(purchase_first_df, on="user_id", how="inner")


In [133]:
selected_level_3["timedelta"] = (
    selected_level_3["purchase_time"] - selected_level_3["level_choice_time"]
)
selected_level_3.head()               # Считаем временную разницу

,user_id,level_choice_time,purchase_time,timedelta
0,27845,2018-01-02 06:19:18,2018-01-03 18:53:43,1 days 12:34:25
1,27865,2018-01-04 05:56:32,2018-01-04 14:46:10,0 days 08:49:38
2,27910,2018-01-05 11:59:50,2018-01-07 12:11:34,2 days 00:11:44
3,27911,2018-01-05 17:39:02,2018-01-07 08:19:12,1 days 14:40:10
4,27940,2018-01-06 00:32:47,2018-01-07 13:16:41,1 days 12:43:54


In [134]:
selected_level_3["timedelta"].describe()

count                          442
mean     3 days 07:20:41.420814479
std      1 days 21:43:52.953292605
min                0 days 03:26:45
25%         1 days 14:57:23.500000
50%         3 days 03:13:57.500000
75%         4 days 19:16:00.250000
max                8 days 01:18:13
Name: timedelta, dtype: object

Среднее время между выбором уровня сложности и покупкой:
- easy - 3 days 14:58:52;
- medium - 3 days 23:14:13;
- hard - 3 days 07:20:41.

Как можнм заметить среднее время между выбором уровня сложности и первой покупкой не сильно отличается по группам. Минимальное время между этими двумя событиями у группы пользователей с выбором уровня сложности hard. Следует также отметить, что у данных пользователей наблюдается минимальное значение максимального временного промежутка - разница с другими двумя уровнями сложностей составляет более двух с половиной суток.

### Расчет временного промежутка между регистрацией и оплатой у групп пользователей с разным уровнем сложности

#### Уровень сложности easy

In [135]:
purchase_easy = purchase_df_easy[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)            # Оставляем только нужные для вычислений столбцы

In [136]:
purchase_reg_easy = purchase_easy.merge(registration_df, on="user_id", how="inner")


In [137]:
purchase_reg_easy["timedelta"] = (
    purchase_reg_easy["purchase_time"] - purchase_reg_easy["registration_time"]
)
purchase_reg_easy.head()               # Считаем временную разницу

,user_id,purchase_time,registration_time,timedelta
0,27884,2018-01-08 19:37:34,2018-01-04 11:50:43,4 days 07:46:51
1,28182,2018-01-12 02:46:01,2018-01-11 10:12:20,0 days 16:33:41
2,28207,2018-01-12 21:00:24,2018-01-11 16:27:37,1 days 04:32:47
3,28090,2018-01-15 23:42:55,2018-01-09 19:31:24,6 days 04:11:31
4,28378,2018-01-18 02:11:41,2018-01-15 14:55:57,2 days 11:15:44


In [138]:
purchase_reg_easy["timedelta"].describe()

count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
Name: timedelta, dtype: object

#### Уровень сложности medium

In [139]:
purchase_medium = purchase_df_medium[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)            # Оставляем только нужные для вычислений столбцы

In [140]:
purchase_reg_medium = purchase_medium.merge(registration_df, on="user_id", how="inner")


In [141]:
purchase_reg_medium["timedelta"] = (
    purchase_reg_medium["purchase_time"] - purchase_reg_medium["registration_time"]
)
purchase_reg_medium.head()               # Считаем временную разницу

,user_id,purchase_time,registration_time,timedelta
0,27981,2018-01-07 23:20:25,2018-01-07 08:09:09,0 days 15:11:16
1,28026,2018-01-09 20:51:27,2018-01-08 13:46:31,1 days 07:04:56
2,28010,2018-01-10 05:32:47,2018-01-07 22:19:23,2 days 07:13:24
3,28094,2018-01-10 13:44:50,2018-01-09 21:25:14,0 days 16:19:36
4,28035,2018-01-10 13:59:12,2018-01-08 17:49:55,1 days 20:09:17


In [142]:
purchase_reg_medium["timedelta"].describe()

count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
Name: timedelta, dtype: object

#### Уровень сложности hard

In [143]:
purchase_hard = purchase_df_hard[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)            # Оставляем только нужные для вычислений столбцы

In [144]:
purchase_reg_hard = purchase_hard.merge(registration_df, on="user_id", how="inner")


In [145]:
purchase_reg_hard["timedelta"] = (
    purchase_reg_hard["purchase_time"] - purchase_reg_hard["registration_time"]
)
purchase_reg_hard.head()               # Считаем временную разницу

,user_id,purchase_time,registration_time,timedelta
0,27845,2018-01-03 18:53:43,2018-01-02 01:35:56,1 days 17:17:47
1,27865,2018-01-04 14:46:10,2018-01-03 11:14:57,1 days 03:31:13
2,27911,2018-01-07 08:19:12,2018-01-05 10:48:24,1 days 21:30:48
3,27910,2018-01-07 12:11:34,2018-01-05 10:45:33,2 days 01:26:01
4,27940,2018-01-07 13:16:41,2018-01-05 23:41:24,1 days 13:35:17


In [146]:
purchase_reg_hard["timedelta"].describe()

count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
Name: timedelta, dtype: object

Среднее время между регистрацией и оплатой по уровням сложности:
- easy - 3 days 22:10:23;
- medium - 4 days 06:12:06;
- hard - 3 days 14:55:19.

Как и в предыдущих данных среднее время между регистрацией и первой покупкой минимально у группы пользователей с выбором уровня сложности hard. Самое большре значение у группы пользователей с уровнем сложности medium. Следует также отметить, что у группы пользователей с выбором уровня сложности hard также наблюдается разница с другими двумя уровнями сложностей более двух с половиной суток в значении максимального временного промежутка.

Из приведённых данных можно сделать вывод, что группы пользователей с высоким уровнем сложности более расположены к покупкам